In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# đọc các file csv lên
books = pd.read_csv('../input/databooks/dataset/books.csv')
ratings = pd.read_csv('../input/databooks/dataset/ratings.csv')
book_tags = pd.read_csv('../input/databooks/dataset/book_tags.csv')
tags = pd.read_csv('../input/databooks/dataset/tags.csv')

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import  cosine_similarity

In [ ]:
#danh sách tác giả (chuyển thành chữ thường, remove khoảng trắng, tách danh sách dựa vào dấu , )
books['authors'] = books['authors'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x.split(', ')])

In [ ]:
def get_genres(x):
    t = book_tags[book_tags.goodreads_book_id==x]
    return [i.lower().replace(" ", "") for i in tags.tag_name.loc[t.tag_id].values]

In [ ]:
# danh sách thể loại
books['genres'] = books.book_id.apply(get_genres)
books['genres']

In [ ]:
#Kết hợp sách với các thể loại tương ứng
books['list'] = books.apply(lambda x: ' '.join([x['title']] + x['authors'] + x['genres']), axis=1)
books.list.head()

In [ ]:
#dùng CountVectorizer và transform text thành vector
count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
count_matrix = count.fit_transform(books['list'])

In [ ]:
# dùng cosine_similarity để tính toán một đại lượng số biểu thị sự giống nhau giữa hai cuốn sách
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [ ]:
indices = pd.Series(books.index, index=books['title'])
titles = books['title']
titles

In [ ]:
# tim tên sách theo từ khóa nếu quên tên sách
def get_name_from_partial(title):
    return list(books.title[books.title.str.lower().str.contains(title) == True].values)

title = "business"
l = get_name_from_partial(title)
list(enumerate(l))

In [ ]:
# xây dựng hàm khuyến nghị 
def get_recommendations(title, n=10):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx])) #dùng enumerate() thêm vào một bộ đếm vào trước mỗi iterable và trả về kết quả dưới dạng đối tượng liệt kê và chuyển thành danh sách
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True) # sắp xếp similarity scores 
    sim_scores = sim_scores[1:31] # top 30 book dựa vào similarity scores 
    book_indices = [i[0] for i in sim_scores]
    return list(titles.iloc[book_indices].values)[:n]

In [ ]:
get_recommendations("The Hunger Games (The Hunger Games, #1)")